# ⼆维卷积层
卷积神经⽹络（convolutional neural network）是含有卷积层（convolutional layer）的神经⽹
络。本章中介绍的卷积神经⽹络均使⽤最常⻅的⼆维卷积层。它有⾼和宽两个空间维度，常⽤来
处理图像数据。本节中，我们将介绍简单形式的⼆维卷积层的⼯作原理。
## ⼆维互相关运算
虽然卷积层得名于卷积（convolution）运算，但我们通常在卷积层中使⽤更加直观的互相关（crosscorrelation）运算。在⼆维卷积层中，⼀个⼆维输⼊数组和⼀个⼆维核（kernel）数组通过互相
关运算输出⼀个⼆维数组。我们⽤⼀个具体例⼦来解释⼆维互相关运算的含义。如图5.1所⽰，输
⼊是⼀个⾼和宽均为3的⼆维数组。我们将该数组的形状记为3 × 3或（3， 3）。核数组的⾼和宽分
别为2。该数组在卷积计算中⼜称卷积核或过滤器（filter）。卷积核窗口（⼜称卷积窗口）的形状
取决于卷积核的⾼和宽，即2 × 2。图5.1中的阴影部分为第⼀个输出元素及其计算所使⽤的输⼊
和核数组元素： 0 × 0 + 1 × 1 + 3 × 2 + 4 × 3 = 19。

![二维互相关运算](../img/correlation.svg)

在⼆维互相关运算中，卷积窗口从输⼊数组的最左上⽅开始，按从左往右、从上往下的顺序，依
次在输⼊数组上滑动。当卷积窗口滑动到某⼀位置时，窗口中的输⼊⼦数组与核数组按元素相乘
并求和，得到输出数组中相应位置的元素。图5.1中的输出数组⾼和宽分别为2，其中的4个元素由
⼆维互相关运算得出：

$\begin{aligned}
0\times0+1\times1+3\times2+4\times3=19,\\
1\times0+2\times1+4\times2+5\times3=25,\\
3\times0+4\times1+6\times2+7\times3=37,\\
4\times0+5\times1+7\times2+8\times3=43.\\
\end{aligned}
$

下⾯我们将上述过程实现在corr2d函数⾥。它接受输⼊数组X与核数组K，并输出数组Y。

In [4]:
from mxnet import autograd, nd
from mxnet.gluon import nn
def corr2d(X,K):
    h,w=K.shape
    Y = nd.zeros(shape=(X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i][j] = (X[i: i+h, j: j+w] * K).sum()
    return Y

我们可以构造图5.1中的输⼊数组X、核数组K来验证⼆维互相关运算的输出。

In [9]:
X = nd.reshape(nd.array(range(9)),shape=(3,3))
K = nd.reshape(nd.array(range(4)),shape=(2,2))
Y = corr2d(X,K)
Y


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @cpu(0)>

## ⼆维卷积层
⼆维卷积层将输⼊和卷积核做互相关运算，并加上⼀个标量偏差来得到输出。卷积层的模型参数
包括了卷积核和标量偏差。在训练模型的时候，通常我们先对卷积核随机初始化，然后不断迭代
卷积核和偏差。

下⾯基于corr2d函数来实现⼀个⾃定义的⼆维卷积层。在构造函数__init__⾥我们声
明weight和bias这两个模型参数。前向计算函数forward则是直接调⽤corr2d函数再加上偏差

In [ ]:
class Conv2D(nn.Block):
    def __init__(self, kernel_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        self.weight = self.params.get('weight',shape=kernel_size)
        self.bias = self.params.get('bias', shape=(1,))
    def forward(self, data):
        return corr2d(data, self.weight.data()) + self.bias.data()

卷积窗口形状为$p×q$的卷积层称为$p×q$卷积层。同样，$p×q$卷积或$p×q$卷积核说明卷积核的⾼
和宽分别为$p和q$
## 图像中物体边缘检测
下⾯我们来看⼀个卷积层的简单应⽤：检测图像中物体的边缘，即找到像素变化的位置。⾸先我
们构造⼀张6 × 8的图像（即⾼和宽分别为6像素和8像素的图像）。它中间4列为⿊（0），其余为⽩

In [11]:
X = nd.ones(shape=(6,8))
X[:,2:6] = 0
X


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

然后我们构造⼀个⾼和宽分别为1和2的卷积核K。当它与输⼊做互相关运算时，如果横向相邻元
素相同，输出为0；否则输出为⾮0。

In [12]:
K = nd.array([[1,-1]])

下⾯将输⼊X和我们设计的卷积核K做互相关运算。可以看出，我们将从⽩到⿊的边缘和从⿊到⽩
的边缘分别检测成了1和-1。其余部分的输出全是0。

In [13]:
Y = corr2d(X,K)
Y


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

由此，我们可以看出，卷积层可通过重复使⽤卷积核有效地表征局部空间。
## 通过数据学习核数组
最后我们来看⼀个例⼦，它使⽤物体边缘检测中的输⼊数据X和输出数据Y来学习我们构造的核
数组K。我们⾸先构造⼀个卷积层，将其卷积核初始化成随机数组。接下来在每⼀次迭代中，我
们使⽤平⽅误差来⽐较Y和卷积层的输出，然后计算梯度来更新权重。简单起⻅，这⾥的卷积层
忽略了偏差。

虽然我们之前构造了Conv2D类，但由于corr2d使⽤了对单个元素赋值（[i, j]=）的操作因而
⽆法⾃动求梯度。下⾯我们使⽤Gluon提供的Conv2D类来实现这个例⼦。

In [14]:
# 构造⼀个输出通道数为1（将在“多输⼊通道和多输出通道”⼀节介绍通道），核数组形状是(1, 2)的⼆维卷积层
conv2d = nn.Conv2D(1,kernel_size=(1,2))
conv2d.initialize()
# ⼆维卷积层使⽤4维输⼊输出，格式为(样本, 通道, ⾼, 宽)，这⾥批量⼤⼩（批量中的样本数）和通
# 道数均为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
for i in range(10):
    with autograd.record():
        Y_hat = conv2d(X)
        l = (Y_hat - Y) ** 2
    l.backward()
    # 简单起⻅，这⾥忽略了偏差
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
    if (i + 1) % 2 == 0:
        print('batch %d, loss %.3f' % (i + 1, l.sum().asscalar()))

batch 2, loss 4.949
batch 4, loss 0.831
batch 6, loss 0.140
batch 8, loss 0.024
batch 10, loss 0.004


可以看到， 10次迭代后误差已经降到了⼀个⽐较小的值。现在来看⼀下学习到的核数组。

In [16]:
conv2d.weight.data().reshape(1,2)


[[ 0.9895    -0.9873705]]
<NDArray 1x2 @cpu(0)>

可以看到，学到的核数组与我们之前定义的核数组K较接近。
## 互相关运算和卷积运算
实际上，卷积运算与互相关运算类似。为了得到卷积运算的输出，我们只需将核数组左右翻转并
上下翻转，再与输⼊数组做互相关运算。可⻅，卷积运算和互相关运算虽然类似，但如果它们使
⽤相同的核数组，对于同⼀个输⼊，输出往往并不相同。

那么，你也许会好奇卷积层为何能使⽤互相关运算替代卷积运算。其实，在深度学习中核数组都
是学出来的：卷积层⽆论使⽤互相关运算或卷积运算都不影响模型预测时的输出。为了解释这⼀
点，假设卷积层使⽤互相关运算学出图5.1中的核数组。设其他条件不变，使⽤卷积运算学出的核
数组即图5.1中的核数组按上下、左右翻转。也就是说，图5.1中的输⼊与学出的已翻转的核数组
再做卷积运算时，依然得到图5.1中的输出。为了与⼤多数深度学习⽂献⼀致，如⽆特别说明，本
书中提到的卷积运算均指互相关运算。
## 特征图和感受野
⼆维卷积层输出的⼆维数组可以看作是输⼊在空间维度（宽和⾼）上某⼀级的表征，也叫特征图
（feature map）。影响元素x的前向计算的所有可能输⼊区域（可能⼤于输⼊的实际尺⼨）叫做x的
感受野（receptive field）。以图5.1为例，输⼊中阴影部分的四个元素是输出中阴影部分元素的感
受野。我们将图5.1中形状为2 × 2的输出记为Y ，并考虑⼀个更深的卷积神经⽹络：将Y 与另⼀个
形状为2 × 2的核数组做互相关运算，输出单个元素z。那么， z在Y 上的感受野包括Y 的全部四个
元素，在输⼊上的感受野包括其中全部9个元素。可⻅，我们可以通过更深的卷积神经⽹络使特
征图中单个元素的感受野变得更加⼴阔，从而捕捉输⼊上更⼤尺⼨的特征。

我们常使⽤“元素”⼀词来描述数组或矩阵中的成员。在神经⽹络的术语中，这些元素也可称为
“单元”。当含义明确时，本书不对这两个术语做严格区分。
## 小结
- ⼆维卷积层的核⼼计算是⼆维互相关运算。在最简单的形式下，它对⼆维输⼊数据和卷积
核做互相关运算然后加上偏差。
- 我们可以设计卷积核来检测图像中的边缘。
- 我们可以通过数据来学习卷积核。